**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [6]:
import tensorflow as tf

이 노트북에서는 하나 이상의 GPU에서 모델을 실행하거나 훈련하는 방법을 수행하므로 적어도 하나 이상의 GPU가 있는지 확인하거나 그렇지 않으면 경고를 발행한다:

In [7]:
if not tf.config.list_physical_devices('GPU'):
    print('GPU가 감지되지 않았습니다. 신경망은 GPU가 없으면 매우 느릴 수 있습니다.')

2024-02-25 09:34:16.284679: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:34:16.535924: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:34:16.536173: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.


# 텐서플로 모델 서빙하기

# 대규모 텐서플로 모델 훈련과 배포

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포한다.

## 텐서플로 서빙 사용

가장 먼저 해야 할 일은 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보내는 것이다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [9]:
from pathlib import Path

# MNIST 데이터 세트 로드 및 분할
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
# MNIST 모델 구축 및 훈련(이미지 전처리도 처리)
tf.keras.backend.clear_session()
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
        tf.keras.layers.Rescaling(1 / 255),
        tf.keras.layers.Dense(100, 'relu'),
        tf.keras.layers.Dense(10, 'softmax')
    ]
)
model.compile(tf.keras.optimizers.experimental.SGD(1e-2), 'sparse_categorical_crossentropy', ['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model_name = 'my_mnist_model'
model_version = '0001'
model_path = Path(model_name) / model_version
model.save(model_path, save_format='tf')

2024-02-25 09:43:07.873967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:43:07.874316: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:43:07.874520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:43:08.034058: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:43:08.034314: I tensorflow/compile

Epoch 1/10


2024-02-25 09:43:09.309192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-25 09:43:09.502211: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f27d083a110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-25 09:43:09.502251: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-02-25 09:43:09.654228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2024-02-25 09:43:09.909859: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-25 09:43:10.095065: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1719/1719 [==============================] - 6s 3ms/step - loss: 0.6816 - accuracy: 0.8304 - val_loss: 0.3677 - val_accuracy: 0.8998
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3493 - accuracy: 0.9031 - val_loss: 0.2992 - val_accuracy: 0.9160
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3001 - accuracy: 0.9161 - val_loss: 0.2643 - val_accuracy: 0.9288
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2691 - accuracy: 0.9239 - val_loss: 0.2417 - val_accuracy: 0.9312
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2455 - accuracy: 0.9313 - val_loss: 0.2228 - val_accuracy: 0.9382
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2268 - accuracy: 0.9364 - val_loss: 0.2075 - val_accuracy: 0.9404
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2110 - accuracy: 0.9401 - val_loss: 0.1942 - val_accuracy: 0.9470
Epo

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


파일 트리를 살핀다:

In [10]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

SavedModel을 검사한다:

In [16]:
!saved_model_cli show --dir '{model_path}'

2024-02-25 09:55:18.828532: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-25 09:55:20.010246: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:55:20.025038: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:55:20.025256: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been bu

In [17]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2024-02-25 09:55:52.834180: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-25 09:55:54.026782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:55:54.040408: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:55:54.040778: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been bu

In [18]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2024-02-25 09:56:21.276452: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-25 09:56:22.441738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:56:22.456355: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-25 09:56:22.456633: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been bu

---

In [8]:
for root, _, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print(f'{indent}{os.path.basename(root)}/')
    for filename in files:
        print(f'{indent}    {filename}')

my_mnist_model/
    0001/
        fingerprint.pb
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index


In [9]:
!saved_model_cli show --dir {model_path}

2023-08-21 08:15:39.794518: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 08:15:41.048459: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 08:15:41.073743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 08:15:41.074078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been bu

In [10]:
!saved_model_cli show --dir {model_path} --tag_set serve

2023-08-21 08:15:42.674465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 08:15:43.816225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 08:15:43.832320: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 08:15:43.832573: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been bu

In [11]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2023-08-21 08:15:44.604485: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 08:15:45.993620: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 08:15:46.015223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-21 08:15:46.015645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been bu

더 자세한 내용을 보려면 다음 명령을 실행한다:

```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

이 노트북을 실행하는 경우, 텐서플로 서버를 설치해야 한다:

In [29]:
url = 'https://storage.googleapis.com/tensorflow-serving-apt'
src = 'stable tensorflow-model-server tensorflow-model-server-universal'
!echo 'deb {url} {src}' > /etc/apt/sources.list.d/tensorflow-serving.list
!curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
!apt update -q && apt-get install -y tensorflow-model-server
%pip install -q -U tensorflow-serving-api

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  32340      0 --:--:-- --:--:-- --:--:-- 32340
OK
Hit:1 https://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Building dependency tree...
Reading state information...
All packages are up to date.
Reading package lists... D

`tensorflow_model_server`가 설치된 경우 다음 2개의 셀을 실행하여 서버를 시작한다.

In [30]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [35]:
%%bash --bg
tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=my_mnist_model --model_base_path="${MODEL_DIR}" &> my_server.log

도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행한다. `path/to/my_mnist_model`을 `my_mnist_model` 디렉토리의 적절한 절대 경로로 대체해야 하지만, 컨테이너 경로 `/models/my_mnist_model`은 수정하지 않는다.
```bash
docker pull tensorflow/serving  # 최신 TF 서빙 이미지 다운로드
docker run -it --rm -v "/path/to/my_mnist_model:/models/my_mnist_model" -p 8500:8500 -p 8501:8501 -e MODEL_NAME=my_mnist_model tensorflow/serving
```

---

**REST API로 TF 서빙에 쿼리하기**

In [16]:
import json

input_data_json = json.dumps({"signature_name": "serving_default", "instances": X_new.tolist()})

In [17]:
repr(input_data_json)[:1500] + '...'

'\'{"signature_name": "serving_default", "instances": [[[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0

이제 텐서플로 서빙의 REST API를 사용해 예측을 만든다:

In [18]:
import requests

SERVER_URL = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, input_data_json)
response.raise_for_status()  # 에러가 생길 경우 예외를 발생한다.
response = response.json()

In [19]:
response.keys()

dict_keys(['predictions'])

In [20]:
y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.01],
       [0.05, 0.  , 0.74, 0.03, 0.  , 0.08, 0.08, 0.  , 0.02, 0.  ],
       [0.  , 0.94, 0.02, 0.01, 0.  , 0.  , 0.01, 0.01, 0.01, 0.  ]])

**gRPC API로 TF 서빙에 쿼리하기**

In [21]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [22]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('172.17.0.3:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.)

In [23]:
response

model_spec {
  name: "my_mnist_model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}
outputs {
  key: "dense_1"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 3
      }
      dim {
        size: 10
      }
    }
    float_val: 0.000402979786
    float_val: 5.88208832e-06
    float_val: 0.000366911176
    float_val: 0.00135380519
    float_val: 0.000246009586
    float_val: 0.00012352725
    float_val: 6.24434233e-06
    float_val: 0.986865819
    float_val: 0.000193533
    float_val: 0.010435285
    float_val: 0.0472884588
    float_val: 0.00201013405
    float_val: 0.741884887
    float_val: 0.0263824854
    float_val: 4.35053189e-05
    float_val: 0.0835964605
    float_val: 0.077808
    float_val: 1.3808849e-05
    float_val: 0.0208080988
    float_val: 0.000164117519
    float_val: 0.000934983371
    float_val: 0.935363352
    float_val: 0.0228319
    float_val: 0.0100065954
    float_val: 0.00207439577
    float_val: 0.00379154854
  

응답을 텐서로 변환한다:

In [24]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.01],
       [0.05, 0.  , 0.74, 0.03, 0.  , 0.08, 0.08, 0.  , 0.02, 0.  ],
       [0.  , 0.94, 0.02, 0.01, 0.  , 0.  , 0.01, 0.01, 0.01, 0.  ]],
      dtype=float32)

클라이언트가 텐서플로 라이브러리를 사용하지 않는다면 넘파이 배열로 변환한다:

In [25]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.01],
       [0.05, 0.  , 0.74, 0.03, 0.  , 0.08, 0.08, 0.  , 0.02, 0.  ],
       [0.  , 0.94, 0.02, 0.01, 0.  , 0.  , 0.01, 0.01, 0.01, 0.  ]])

**새로운 버전의 모델 배포하기**

In [26]:
model = keras.Sequential(
    [
        keras.layers.Flatten(input_shape=[28, 28, 1]),
        keras.layers.Dense(50, 'relu'),
        keras.layers.Dense(50, 'relu'),
        keras.layers.Dense(10, 'softmax')
    ]
)
model.compile(keras.optimizers.experimental.SGD(), 'sparse_categorical_crossentropy', ['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.7323 - accuracy: 0.7955 - val_loss: 0.3438 - val_accuracy: 0.9082
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3241 - accuracy: 0.9070 - val_loss: 0.2710 - val_accuracy: 0.9276
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2712 - accuracy: 0.9218 - val_loss: 0.2377 - val_accuracy: 0.9352
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2388 - accuracy: 0.9307 - val_loss: 0.2107 - val_accuracy: 0.9418
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2138 - accuracy: 0.9381 - val_loss: 0.1985 - val_accuracy: 0.9454
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.1950 - accuracy: 0.9436 - val_loss: 0.1762 - val_accuracy: 0.9544
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.1777 - accuracy: 0.9480 - val_loss: 0.1661 - val_accuracy:

In [27]:
model_version = '0002'
model_name = 'my_mnist_model'
model_path = os.path.join(model_name, model_version)
model_path

'my_mnist_model/0002'

In [28]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


In [29]:
for root, _, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print(f'{indent}{os.path.basename(root)}/')
    for filename in files:
        print(f'{indent}    {filename}')

my_mnist_model/
    0001/
        fingerprint.pb
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index
    0002/
        fingerprint.pb
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index


**경고**: 새로운 모델이 텐서플로 서빙에 로드되기 전까지 잠시 기다려야 할 수 있다:

In [30]:
SERVER_URL = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, input_data_json)
response.raise_for_status()
response = response.json()

In [31]:
response.keys()

dict_keys(['predictions'])

In [32]:
y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.01],
       [0.05, 0.  , 0.74, 0.03, 0.  , 0.08, 0.08, 0.  , 0.02, 0.  ],
       [0.  , 0.94, 0.02, 0.01, 0.  , 0.  , 0.01, 0.01, 0.01, 0.  ]])

### 예측 서비스 사용하기

**구글 클라우드 클라이언트 라이브러리**

구글 클라우드 AI 플랫폼에 모델을 배포하고, 서비스 계정의 개인키를 다운로드하여 프로젝트 디렉토리에 저장한다:

In [33]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "mineral-trainer-394903-4289c79f86b1.json"

In [34]:
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
        project, endpoint_id, instances, location='us-central1', api_endpoint='us-central1-aiplatform.googleapis.com'
):
    client_options = {'api_endpoint': api_endpoint}
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    instances = instances if isinstance(instances, list) else [instances]
    instances = [json_format.ParseDict(instance_dict, Value()) for instance_dict in instances]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(project=project, location=location, endpoint=endpoint_id)
    response = client.predict(endpoint=endpoint, instances=instances, parameters=parameters)
    predictions = response.predictions
    return np.array(predictions)

In [35]:
Y_probas = predict_custom_trained_model_sample(
    '759555954445',
    '8228789002740695040',
    X_new.tolist(),
    'asia-northeast3',
    'asia-northeast3-aiplatform.googleapis.com'
)
np.round(Y_probas, 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

# 모바일 또는 임베디드 디바이스에 모델 배포하기

In [4]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open('my_converted_savedmodel.tflite', 'wb') as f:
    f.write(tflite_model)

NameError: name 'model_path' is not defined